Copied over from week 10

In [1]:
from functools import reduce

In [2]:
def apply_jump(jump_size, position, input_list):
    overlap = jump_size - (len(input_list) - position)
    if overlap < 0:
        overlap = 0
    section = list(input_list[position: position + jump_size])
    if jump_size > len(input_list):
        print("Unhandled jump size", jump_size)
    if overlap:
        section += input_list[0: overlap]
    section.reverse()
    
    # overlay reversal on original list
    new_input_list = input_list[0:position]
    new_input_list += section[0:jump_size - overlap]
    if not overlap:
        new_input_list += input_list[position + jump_size:]
    else:
        new_input_list[0:overlap] = section[-overlap:]
    
    return new_input_list
    

In [3]:
def solve(skip, position, buckets, jumps):
    for j in jumps:
        buckets = apply_jump(j, position, buckets)
        position = (position + skip + j) % len(buckets)
        #print(j, position, buckets)
        skip += 1
    return skip, position, buckets

In [4]:
def to_ascii(input_str):
    return [ord(s) for s in input_str] + [17, 31, 73, 47, 23]


In [5]:
def make_dense_hash(sparse_hash):
    res = []
    assert len(sparse_hash) == 256
    for i in range(16):
        res.append(reduce(lambda i, j: i ^ j, sparse_hash[i*16: (i+1) * 16]))
    return res
        

In [6]:
def knot_hash(input_str):
    jumps = to_ascii(input_str)
    position = 0
    skip = 0
    buckets = list(range(256))
    
    for i in range(64):
        skip, position, buckets = solve(skip, position, buckets, jumps)
    
    sparse_hash = buckets
    dense_hash = make_dense_hash(sparse_hash)
    return "".join(["{:#04x}".format(d)[2:] for d in dense_hash])


## Week 14 starts here

```
Continuing this process, the first 8 rows and columns for key flqrgnkx appear as follows, using # to denote used squares, and . to denote free ones:

##.#.#..-->
.#.#.#.#   
....#.#.   
#.#.##.#   
.##.#...   
##..#..#   
.#...#..   
##.#.##.-->
|      |   
V      V
```

In [7]:
knot_hash('flqrgnkx-0')

'd4f76bdcbf838f8416ccfa8bc6d1f9e6'

In [8]:
# Google, cut and paste
binary = lambda x:"".join(reversed( [i+j for i,j in zip( *[ ["{0:04b}".format(int(c,16)) for c in reversed("0"+x)][n::2] for n in [1,0] ] ) ] ))

In [9]:
for i in range(8):
    print(binary(knot_hash('flqrgnkx-{}'.format(i)))[:8])

11010100
01010101
00001010
10101101
01101000
11001001
01000100
11010110


In [10]:
def get_hashes(key):
    hashes = list()
    for i in range(128):
        hashes.append(binary(knot_hash('{}-{}'.format(key, i))))
    return hashes   

In [11]:
def get_used(hashes):
    return sum(h.count('1') for h in hashes)

In [12]:
get_used(get_hashes('flqrgnkx'))

8108

In [13]:
get_used(get_hashes('hwlqcszp'))

8304

In [14]:
def hashes_to_grid(hashes):
    return [list(h) for h in hashes]

In [15]:
def gen_neighbours(x, y, size=128):
    inf_neighbours = (
        # (x+1, y-1),
        (x+1, y),
        # (x+1, y+1),
        (x, y-1),
        (x, y+1),
        # (x-1, y-1),
        (x-1, y),
        # (x-1, y+1),
    )
    return (
        (x, y) for (x, y) in inf_neighbours
        if x >= 0 and x < size and
        y >= 0 and y < size
    )

In [16]:
list(gen_neighbours(12, 12))

[(13, 12), (12, 11), (12, 13), (11, 12)]

In [17]:
def unseen_used_neighbours(grid, x, y):
    return [(x, y) for (x, y) in gen_neighbours(x, y) if grid[x][y] == '1']    

In [18]:
def defrag(grid):
    num_groups = 0
    for x in range(len(grid)):
        for y in range(len(grid)):
            if grid[x][y] != '1':
                continue
            num_groups += 1
            grid[x][y] = '*'
            stack = unseen_used_neighbours(grid, x, y)
            while stack:
                stack_x, stack_y = stack.pop()
                grid[stack_x][stack_y] = '*'
                stack += unseen_used_neighbours(grid, stack_x, stack_y)
    return num_groups       

In [19]:
grid = hashes_to_grid(get_hashes('flqrgnkx'))
defrag(grid)

1242

In [20]:
grid = hashes_to_grid(get_hashes('hwlqcszp'))
defrag(grid)

1018